In [3]:
! pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from rank_bm25 import BM25Okapi

In [5]:
import pandas as pd

df=pd.read_csv('CompleteMainText.csv')
df_just_text=df[['DocID','Text','URL']]
# print(df_just_text)
df_just_text['Text'] = df_just_text['Text'].str.replace("\n"," ")
corpus = df_just_text['Text'].tolist()
print(len(corpus))
# corpus['Text']=corpus['Text'].tolist()

198


<ipython-input-5-a151e8a71717>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_just_text['Text'] = df_just_text['Text'].str.replace("\n"," ")


In [6]:
# df_corpus_token['Text'] = [string.replace("\n"," ") for string in df_just_text['Text']]
# print(df_corpus_token['Text'][0])

In [7]:
tokenized_corpus = [doc.split(" ") for doc in corpus]

In [8]:
df_tokenized_corpus=df_just_text.copy()
df_tokenized_corpus['Text'] = [doc.split(" ") for doc in df_just_text['Text']]
tokenized_corpus = df_tokenized_corpus['Text'].tolist()
# df_tokenized_corpus['Text'][0]
# df_tokenized_corpus['Text'][0]

In [9]:
bm25 = BM25Okapi(tokenized_corpus)

In [10]:
# df_corpus=df_just_text.copy()
# df_corpus_token=df_just_text.copy()
# df_corpus_token['Text'] = [doc.replace("\n"," ").split(" ") for doc in df_corpus['Text']]

In [11]:
# print(df_corpus_token)

In [12]:
# bm25 = BM25Okapi(df_corpus_token['Text'])

In [13]:
query_list = ["fidgety child unable to sit still",
"child is forgetful in daily activities",
"child disregards rules",
"child not falling asleep in own bed",
"bedwetting in school age children",
"my child worries a lot",
"my child argues with me and screams",
"defiant argumentative behavior in child",
"potty training tips for toddlers",
"handling toddler tantrums",
"child is distracted and cannot focus",
"destructive and aggressive behavior in child"]

tokenized_query=[]
for query in query_list:
  tokenized_query.append(query.split(" "))

In [ ]:
# tokenized_query

In [ ]:
# bm25.get_top_n(tokenized_query[0], corpus, n=5)

In [ ]:
# doc_scores = bm25.get_scores(tokenized_query)

In [ ]:
# top_text=bm25.get_top_n(tokenized_query, corpus, n=2)

In [ ]:
# doc_index=df_corpus_token[df_corpus_token['Text'] == s, 'DocID'].iloc[0]

In [14]:

final_document_List=[]
diction={}

for a,single_query in enumerate(tokenized_query):
  top_text=bm25.get_top_n(single_query, corpus, n=5)
  # print(top_text)
  # print(df_corpus_token['Text']==top_text[0])
  # mask = df_corpus_token['Title'].apply(lambda x: set(top_text[0]).issubset(set(x)) if isinstance(x, list) else False)
  # doc_Index = df_corpus_token[mask]['DocID']
  # print(doc_Index)

  if query_list[a] not in diction:
        diction[query_list[a]] = []
  
  doc_scores = bm25.get_scores(single_query)
  final_score=sorted(doc_scores)
  for i,s in enumerate(top_text):
    # print(s)
    # print(df_corpus[df_corpus_token['Text'] == s, 'DocID'].iloc[0])
    index=corpus.index(s)
    # print("IndexValuue is->"+ str(index))
    # print("DFID is ->"+str(df_just_text.iloc[index]['DocID']))
    # index = df_just_text.loc[df_just_text['Text']==s].index.tolist()[0]
    # index = df_just_text.loc[df_just_text['Text']==s]['DocID'].item()

    # print(index)
    doc_index=df_just_text.iloc[index]['DocID']
    url=df_just_text.iloc[index]['URL']
    # if doc_index>20:
    #   doc_index+=2
    # elif doc_index<20 and doc_index>8:
    #   doc_index+=1

    # doc_index=df_corpus_token[df_corpus_token['Text'] == s, 'DocID'].iloc[0]
    # doc_index=df.loc[df['Title'] == title, 'DocID'].iloc[0]
    title=df.iloc[index]['Title']
    # print("Document ID-> "+str(index))
    # print("Title ->"+str(title))
    score=final_score[-(i+1)]
    diction[query_list[a]].append((doc_index,title,score,url))
    # print("Rank"+str(i)+"DocID-> :"+str(index)

print(diction)

{'fidgety child unable to sit still': [(116, "How to Understand Your Child's Temperament - HealthyChildren.org", 9.828414820361512, 'https://healthychildren.org/English/ages-stages/gradeschool/Pages/How-to-Understand-Your-Childs-Temperament.aspx'), (180, "Some Kids Really Are More Difficult—Here's Why It Happens and How to Deal", 9.50575433208952, 'https://www.parents.com/toddlers-preschoolers/discipline/some-kids-really-are-more-difficult-heres-why-it-happens-and-how-to-deal/'), (25, 'Common Time-Out Mistakes and How to Solve Them', 8.977202449465551, 'https://www.parents.com/toddlers-preschoolers/discipline/time-out/common-time-out-mistakes-and-how-to-solve-them/'), (49, 'Toilet Training  (for Parents) - Nemours KidsHealth', 8.58987526267408, 'https://kidshealth.org/en/parents/toilet-teaching.html#catbehavior'), (112, 'Developmental Milestones: 4 to 5 Year Olds - HealthyChildren.org', 8.417528536518333, 'https://healthychildren.org/English/ages-stages/preschool/Pages/Developmental-Mi

In [15]:
# query='bed wetting'
# for t in diction[query]:
#   print('Score-> '+str(t[2]))
#   print('Title-> '+t[1])
#   print('URL-> '+t[3])

In [16]:
final_df = pd.DataFrame(columns=['Query','rank','Title','DocID','score','URL'])
length=1
for qn,query in enumerate(diction):
  rank=1
  for t in diction[query]:
    docID=t[0]
    title=t[1]
    score=t[2]
    url=t[3]
    final_df.loc[len(final_df)] = [query, rank, title, docID, score,url]
    # final_df.append((query,rank,title,docID,score))
    rank+=1
    length+=1

final_df
  


,Query,rank,Title,DocID,score,URL
0,fidgety child unable to sit still,1,How to Understand Your Child's Temperament - H...,116,9.828415,https://healthychildren.org/English/ages-stage...
1,fidgety child unable to sit still,2,Some Kids Really Are More Difficult—Here's Why...,180,9.505754,https://www.parents.com/toddlers-preschoolers/...
2,fidgety child unable to sit still,3,Common Time-Out Mistakes and How to Solve Them,25,8.977202,https://www.parents.com/toddlers-preschoolers/...
3,fidgety child unable to sit still,4,Toilet Training (for Parents) - Nemours KidsH...,49,8.589875,https://kidshealth.org/en/parents/toilet-teach...
4,fidgety child unable to sit still,5,Developmental Milestones: 4 to 5 Year Olds - H...,112,8.417529,https://healthychildren.org/English/ages-stage...
5,child is forgetful in daily activities,1,7 Ways to Encourage Speech from a Speech Langu...,148,11.023383,https://www.mother.ly/toddler/toddler-mileston...
6,child is forgetful in daily activities,2,Dealing With the Terrible 2s: Tantrums,95,10.666877,https://www.webmd.com/parenting/guide/the-drea...
7,child is forgetful in daily activities,3,How to Understand Your Child's Temperament - H...,116,10.400632,https://healthychildren.org/English/ages-stage...
8,child is forgetful in daily activities,4,How to Build Your Child's Confidence and Self-...,68,10.359256,https://childdevelopmentinfo.com/parenting/how...
9,child is forgetful in daily activities,5,Developmental Milestones: 4 to 5 Year Olds - H...,112,10.292839,https://healthychildren.org/English/ages-stage...


Structuring result dictionary to fit into Evaluation Library

In [18]:
# df = pd.DataFrame(final_df)
temp_result={}
result_dict={}
filtered_df=pd.DataFrame(columns=['Query','DocID','score'])
for i,query in enumerate(query_list):
  filtered_df=final_df.loc[final_df['Query'] == query, ['DocID', 'score']]
  # print(final_df.loc[final_df['Query'] == query, ['DocID', 'score']])
  # Assuming the dataframe is named "df"
  docIDs=filtered_df['DocID'].tolist()
  scores=filtered_df['score'].tolist()
  qID="q_"+str(i+1)
  for j,value in enumerate(docIDs):
    dID="d_"+str(value)
    if qID not in result_dict:
      result_dict[qID]={}
      result_dict[qID][dID]=scores[j]
    else:
      result_dict[qID][dID]=scores[j]
  # selected_rows = final_df[final_df['Query'] == query][['DocID', 'score']]
  # Assuming the dataframe is named "selected_rows"
  # selected_rows['DocID'] = selected_rows['DocID'].apply(lambda x: 'd_' + str(x))
  # result_dict = {temp: selected_rows.to_dict('records')}
  # print(filtered_df)

print(filtered_df)
  # temp_result[temp]=


    DocID      score
55    174  10.763920
56     70  10.555352
57      1  10.148365
58     59  10.069589
59    145  10.068893


In [19]:
print(result_dict['q_2'])

{'d_148': 11.023382677522156, 'd_95': 10.666876972547392, 'd_116': 10.400631913337904, 'd_68': 10.35925569754368, 'd_112': 10.292838790307636}


In [20]:
# for_dic = {}
# for i,query in enumerate(query_list):
#   temp="d_"+str(i)
#   for_dic[temp]=


Validation Start

In [21]:
! pip install ranx


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.6/224.6 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyautocorpus: filename=pyautocorpus-0.1.9-cp310-cp310-linux_x86_64.whl size=189824 sha256=38d7f15599271ea3db2f75c3274b3b8c736ec51579479c1f0bf35191d945866a
  Stored in directory: /root/.cache/pip/wheels/e2/be/43/ec1e16468c393f6e474e3be

In [22]:
from ranx import Qrels, Run

qrels_dict = { "q_1": { "d_65":2, "d_69":2, "d_173":2, "d_1":2, "d_41":2, 
                       "d_15": 1, "d_58": 1, "d_66": 1, "d_69": 1, "d_145": 1, "d_173" : 1},
"q_2": { "d_1":2,"d_41":2,"d_65":2,"d_15":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_3": {"d_6":2,"d_17":2,"d_80":2,"d_120":2,"d_122":2,"d_178":2,
      "d_10":1,"d_13":1,"d_14":1,"d_16":1,"d_18":1,"d_19":1,"d_25":1,"d_79":1,"d_86":1,"d_182":1},
"q_4":{"d_32":2,"d_157":2,"d_159":2,"d_162":2,"d_187":2,
      "d_188":1,"d_33":1,"d_34":1,"d_37":1,"d_52":1,"d_62":1,"d_76":1,"d_109":1,"d_163":1,"d_164":1,"d_189":1},
"q_5":{"d_23":2,"d_26":2,"d_71":2,"d_98":2,"d_183":2,"d_186":2,
      "d_22":1,"d_27":1,"d_29":1,"d_49":1,"d_60":1,"d_63":1},
"q_6":{"d_5":2,"d_8":2,"d_9":2,"d_35":2,"d_50":2,"d_52":2,"d_53":2,"d_71":2,"d_90":2,"d_125":2,"d_153":2,"d_192":2,"d_193":2,"d_194":2,
      "d_34":1,"d_45":1,"d_138":1,"d_162":1,"d_168":1},
"q_7":{"d_178":2,"d_4":2,"d_10":2,"d_14":2,"d_80":2,"d_156":2,"d_195":2,
       "d_16":1,"d_19":1,"d_25":1,"d_36":1,"d_48":1,"d_70":1,"d_86":1,"d_91":1,"d_120":1,"d_152":1,"d_167":1,"d_175":1,"d_178":1,"d_179":1
      },
"q_8":{"d_4":2,"d_6":2,"d_13":2,"d_17":2,"d_19":2,"d_178":2,
      "d_18":1,"d_36":1,"d_70":1,"d_79":1},
"q_9":{"d_22":2,"d_27":2,"d_29":2,"d_49":2,"d_60":2,"d_63":2,"d_75":2,"d_84":2,"d_99":2,"d_100":2,"d_101":2,"d_102":2,"d_103":2,"d_104":2,"d_105":2,"d_149":2,"d_150":2,
      "d_23":1,"d_26":1,"d_46":1,"d_61":1,"d_71":1,"d_88":1,"d_89":1,"d_98":1},
"q_10":{"d_91":2,"d_10":2,"d_14":2,"d_195":2,"d_36":2,"d_48":2,"d_140":2,"d_152":2,"d_154":2,"d_156":2,"d_167":2,"d_179":2,"d_195":2,
        "d_16":1,"d_18":1,"d_25":1,"d_42":1,"d_70":1,"d_95":1,"d_175":1},
"q_11":{"d_41":2,"d_65":2,"d_169":2,
      "d_5":1,"d_15":1,"d_24":1,"d_50":1,"d_54":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_12":{"d_70":2,"d_85":2,"d_174":2,"d_177":2,"d_42":2,"d_96":2,
      "d_10":1,"d_16":1,"d_18":1,"d_25":1}      
 }

run_dict = result_dict


qrels = Qrels(qrels_dict)
run = Run(run_dict)

In [23]:
from ranx import evaluate

# Compute score for a single metric
# evaluate(qrels, run, "ndcg@5")
final_result=evaluate(qrels, run, ["map", "ndcg"])

# Compute scores for multiple metrics at once
# evaluate(qrels, run, ["map@5", "mrr"])
print(final_result)

{'map': 0.10778508771929825, 'ndcg': 0.2394806735442755}


In [ ]:
final_df.to_csv('BM25Result.csv')

Validation END!!!!!!!!!!!!!!!!!!!!